# CS445 Final Project - Seathru

In [ ]:
import collections
import sys
import argparse
import numpy as np
import sklearn as sk
import scipy as sp
import scipy.optimize
import scipy.stats
import math
from PIL import Image
import rawpy
import matplotlib
from matplotlib import pyplot as plt
from skimage import exposure
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle, estimate_sigma
from skimage.morphology import closing, opening, erosion, dilation, disk, diamond, square

In [ ]:
# maybe set up our image here? except i don't know which image to use

In [ ]:
def find_backscatter_estimation_points(img, depths, num_bins=10, fraction=0.01, max_vals=20, min_depth_percent=0.0):
    # for you
    return None

In [ ]:
def estimate_illumination(img, backscatter, neighborhood_map, num_neighborhoods, p=0.5, f=2.0, max_iters=100, tol=1E-5):
    D = img - backscatter # I = D + B
    avg_cs, avg_cs_prime = np.zeros(img.shape), np.zeros(img.shape) # change type?

    neighborhood_sizes = np.zeros(num_neighborhoods)
    neighborhood_locs = [None] * num_neighborhoods

    for i in range(num_neighborhoods):
        neighborhood_locs[i] = np.where((i+1) == neighborhood_map)
        neighborhood_sizes[i] = np.size(neighborhood_locs[i][0])

    # update average local contrasts
    for i in range(max_iters):
        for j in range(num_neighborhoods):
            locs = neighborhood_locs[i]
            size = neighborhood_sizes[i]
            avg_cs_prime[locs] = (1/size) * (np.sum(avg_cs[locs])-avg_cs[locs])
        updated_avg = (D*p) + (avg_cs_prime*(1-p))
        abs_diff = np.abs(avg_cs-avg_cs_prime)
        if np.max(abs_diff) < tol:
            break
        avg_cs = updated_avg
    
    result = f * denoise_bilateral(np.maximum(0, avg_cs))
    return result